In [ ]:
!pip install requests
!pip install requests beautifulsoup4
!pip install google-cloud-storage
!pip install pandas
!pip install google-cloud-secret-manager
!pip install python-dotenv

In [5]:
import requests, re, io, json, os
import pandas as pd

from bs4 import BeautifulSoup
from google.cloud import storage
from math import log
from google.cloud import secretmanager
from dotenv import load_dotenv

In [2]:
def get_secret_manager_key(project_id, secret_id):
    client = secretmanager.SecretManagerServiceClient()
    name = f"projects/{project_id}/secrets/{secret_id}/versions/latest"
    response = client.access_secret_version(request={"name": name})
 
    payload = response.payload.data.decode("UTF-8")
    return payload

def download_bls_data(bls_url, sub_url_dir, headers):
    url = f'{bls_url}/{sub_url_dir}/'
    # Add User-Agent with email for contact per BLS access policy

    # 1. Fetch the directory page
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)

    # 2. Parse the HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    files_dir = []
    for link in soup.find_all('a'):
        file_link = link.get('href')
        # file_name = file_link.split('/')[-1]
        # if re.search('pr.', file_name): 
        files_dir.append(file_link)
    return files_dir

def sizeof_fmt(num):
    """Human friendly file size"""
    unit_list = list(zip(['bytes', 'kB', 'MB', 'GB', 'TB', 'PB'], [0, 0, 1, 2, 2, 2]))
 
    if num > 1:
        exponent = min(int(log(num, 1024)), len(unit_list) - 1)
        quotient = float(num) / 1024**exponent
        unit, num_decimals = unit_list[exponent]
        format_string = '{:.%sf} {}' % (num_decimals)
        return format_string.format(quotient, unit)
    if num == 0:
        return '0 bytes'
    if num == 1:
        return '1 byte'  

In [8]:
load_dotenv()

project_id = os.getenv("GCP_PROJECT_ID")
email_add = get_secret_manager_key(project_id, 'email_add')

In [9]:
bls_url = "https://download.bls.gov/pub/time.series"
user_agent_value = f"Python Script ({email_add})"
headers = {
    "User-Agent": user_agent_value
}

In [10]:

unique_filenames_dir = download_bls_data(bls_url, 'pr', headers)

In [11]:
pr_fn = 'pr.data.0.Current'
for each_file in unique_filenames_dir:
    if re.search(pr_fn, each_file):
        file_url = f'{bls_url}/pr/{pr_fn}'
        file_response = requests.get(file_url, headers=headers)
        file_response.raise_for_status()
        file_data = file_response.content
        decoded_data = file_data.decode('utf-8')
        df_p1 = pd.read_csv(io.StringIO(decoded_data), sep='\t')

In [12]:
# Strip whitespace from column names
df_p1.columns = df_p1.columns.str.strip()
df_p1['value'] = pd.to_numeric(df_p1['value'], errors='coerce')

# Strip whitespace from column values
df_p1['series_id'] = df_p1['series_id'].str.strip()
df_p1.to_csv('df_p1.txt', index=False)
df_p1
# df_p1.to_csv('df_p1.ipynb', index=False)

,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.600,NaN
1,PRS30006011,1995,Q02,2.100,NaN
2,PRS30006011,1995,Q03,0.900,NaN
3,PRS30006011,1995,Q04,0.100,NaN
4,PRS30006011,1995,Q05,1.400,NaN
...,...,...,...,...,...
36997,PRS88003203,2024,Q02,116.544,NaN
36998,PRS88003203,2024,Q03,116.593,NaN
36999,PRS88003203,2024,Q04,116.682,R
37000,PRS88003203,2024,Q05,116.686,R


In [13]:
# def fetch_data_and_upload_to_gcs(nation_or_state, year):
def fetch_data_and_upload_to_gcs():
    # The API endpoint provided
    # url = "https://honolulu-api.datausa.io/tesseract/data.jsonrecords?cube=acs_yg_total_population_1&drilldowns=Year%2CNation&locale=en&measures=Population"
    url = f"https://honolulu-api.datausa.io/tesseract/data.jsonrecords?cube=acs_yg_total_population_1&drilldowns=Year%2CNation&locale=en&measures=Population"

    # fn = 'population_data.json'

    #--- Fetch the data from the API ---#
    print("Fetching data from API...")
    response = requests.get(url)
    response.raise_for_status()
    api_data = response.json() # Get the data as a Python dictionary
    fmt_file_size = sizeof_fmt(len(api_data))
    print(f"✅ Data fetched successfully with [{fmt_file_size}].")

    # #--- Convert the Python dictionary to a JSON formatted string ---#
    # json_string = json.dumps(api_data, indent=4)

    return api_data

In [14]:
api_data = fetch_data_and_upload_to_gcs()
print(type(api_data))
print(api_data['data'])


Fetching data from API...
✅ Data fetched successfully with [4 bytes].
<class 'dict'>
[{'Nation ID': '01000US', 'Nation': 'United States', 'Year': 2013, 'Population': 316128839.0}, {'Nation ID': '01000US', 'Nation': 'United States', 'Year': 2014, 'Population': 318857056.0}, {'Nation ID': '01000US', 'Nation': 'United States', 'Year': 2015, 'Population': 321418821.0}, {'Nation ID': '01000US', 'Nation': 'United States', 'Year': 2016, 'Population': 323127515.0}, {'Nation ID': '01000US', 'Nation': 'United States', 'Year': 2017, 'Population': 325719178.0}, {'Nation ID': '01000US', 'Nation': 'United States', 'Year': 2018, 'Population': 327167439.0}, {'Nation ID': '01000US', 'Nation': 'United States', 'Year': 2019, 'Population': 328239523.0}, {'Nation ID': '01000US', 'Nation': 'United States', 'Year': 2021, 'Population': 331893745.0}, {'Nation ID': '01000US', 'Nation': 'United States', 'Year': 2022, 'Population': 333287562.0}, {'Nation ID': '01000US', 'Nation': 'United States', 'Year': 2023, 'P

In [15]:
df_p2 = pd.DataFrame(api_data['data'])

# Strip whitespace from column names
df_p2.columns = df_p2.columns.str.strip()

df_p2.to_csv('df_p2.txt', index=False)
df_p2

,Nation ID,Nation,Year,Population
0,01000US,United States,2013,316128839.0
1,01000US,United States,2014,318857056.0
2,01000US,United States,2015,321418821.0
3,01000US,United States,2016,323127515.0
4,01000US,United States,2017,325719178.0
5,01000US,United States,2018,327167439.0
6,01000US,United States,2019,328239523.0
7,01000US,United States,2021,331893745.0
8,01000US,United States,2022,333287562.0
9,01000US,United States,2023,334914896.0


In [ ]:
filtered_df = df_p2[(df_p2['Year'] >= 2013) & (df_p2['Year'] <= 2018)]

#--- Calculate the mean and standard deviation ---#
population_mean = filtered_df['Population'].mean()
population_std = filtered_df['Population'].std()

print(f"Mean Population:     {population_mean:,.0f}")
print(f"Standard Deviation:  {population_std:,.0f}")

with open('population_stats.txt', 'w') as f:
    print(f"Mean Population:     {population_mean:,.0f}", file=f)
    print(f"Standard Deviation:  {population_std:,.0f}", file=f)

Mean Population:     322,069,808
Standard Deviation:  4,158,441


In [17]:
groupedby_dfp1 = df_p1.groupby(["series_id", "year"])["value"].sum().reset_index()
groupedby_dfp1

,series_id,year,value
0,PRS30006011,1995,7.100
1,PRS30006011,1996,-0.500
2,PRS30006011,1997,4.400
3,PRS30006011,1998,4.200
4,PRS30006011,1999,-7.700
...,...,...,...
8557,PRS88003203,2021,523.634
8558,PRS88003203,2022,562.520
8559,PRS88003203,2023,577.546
8560,PRS88003203,2024,583.441


In [19]:
best_years = groupedby_dfp1.loc[groupedby_dfp1.groupby("series_id")["value"].idxmax()]

print(best_years)
# best_years['series_id'] = best_years['series_id'].str.strip()
best_years.to_csv('best_years.txt', index=False)

        series_id  year    value
27    PRS30006011  2022   20.500
58    PRS30006012  2022   17.100
65    PRS30006013  1998  705.895
108   PRS30006021  2010   17.700
139   PRS30006022  2010   12.400
...           ...   ...      ...
8414  PRS88003192  2002  282.800
8467  PRS88003193  2024  860.838
8496  PRS88003201  2022   37.200
8527  PRS88003202  2022   28.700
8560  PRS88003203  2024  583.441

[282 rows x 3 columns]


In [ ]:
# df_p1.columns = df_p1.columns.str.strip()
# df_p2.columns = df_p2.columns.str.strip()
# print("Time series columns:", df_p1.columns.tolist())
# print("Population columns:", df_p2.columns.tolist())

# # df_p1.columns = df_p1.columns.str.lower()
# # df_p2.columns = df_p2.columns.str.lower()
# print("Time series columns:", df_p1.columns.tolist())
# print("Population columns:", df_p2.columns.tolist())

In [20]:
filteredby = df_p1[(df_p1['series_id'] == 'PRS30006032') & (df_p1['period'] == 'Q01')]
filteredby

,series_id,year,period,value,footnote_codes
1057,PRS30006032,1995,Q01,0.0,NaN
1062,PRS30006032,1996,Q01,-4.2,NaN
1067,PRS30006032,1997,Q01,2.8,NaN
1072,PRS30006032,1998,Q01,0.9,NaN
1077,PRS30006032,1999,Q01,-4.1,NaN
1082,PRS30006032,2000,Q01,0.5,NaN
1087,PRS30006032,2001,Q01,-6.3,NaN
1092,PRS30006032,2002,Q01,-6.6,NaN
1097,PRS30006032,2003,Q01,-5.7,NaN
1102,PRS30006032,2004,Q01,2.0,NaN


In [21]:
df_p2.columns = df_p2.columns.str.lower()

results = pd.merge(filteredby, df_p2, on='year', how='inner')
final_report = results[['series_id', 'year', 'period', 'value', 'population']]
print(final_report)
final_report.to_csv('final_report.txt', index=False)

     series_id  year period  value   population
0  PRS30006032  2013    Q01    0.5  316128839.0
1  PRS30006032  2014    Q01   -0.1  318857056.0
2  PRS30006032  2015    Q01   -1.7  321418821.0
3  PRS30006032  2016    Q01   -1.4  323127515.0
4  PRS30006032  2017    Q01    0.9  325719178.0
5  PRS30006032  2018    Q01    0.5  327167439.0
6  PRS30006032  2019    Q01   -1.6  328239523.0
7  PRS30006032  2021    Q01    0.7  331893745.0
8  PRS30006032  2022    Q01    5.3  333287562.0
9  PRS30006032  2023    Q01    0.3  334914896.0
